In [3]:
import requests
from lxml import etree

if __name__ == '__main__':
    session = requests.Session()
    detail_url = 'https://wlkc.ouc.edu.cn'
    # 使用现成的cookie直接绕过登录页面
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36',
        'Cookie': 'JSESSIONID=D7F909CFC0867C34B637056F90EF6F82; LOGIN=3139303230303331303330; UM_distinctid=17bcb2f4e579ab-0e328a4e032f8-a7d193d-144000-17bcb2f4e58315; COMPANY_ID=10122; session_id=C5213DCE7771258BD6D197CB9F9EF42E; s_session_id=29620728AB303E7B331F423AC2F6311B; web_client_cache_guid=96d181ba-548a-4b74-8724-97e4c2772026; ID=444a594d59682f527a70303d; PASSWORD=5569576f51734a6670786b3d; SCREEN_NAME=4f566b646b73396b79642f396274486b665a503656773d3d',
    }
    # 总尝试页面
    total_url = 'https://wlkc.ouc.edu.cn/webapps/gradebook/do/student/viewAttempts?method=list&course_id=_13492_1&outcome_definition_id=_94418_1&outcome_id=_2037807_1'
    page_text_tot = session.get(url=total_url, headers=headers)
    tree_tot = etree.HTML(page_text_tot.text)
    attempt_list = tree_tot.xpath('//div[@class=" columnStep clearfix"]//a/@href')
    # print(attempt_list)

    # 为了去重使用字典保存键值对
    danxuan = {}
    duoxuan = {}
    panduan = {}
    cnt1 = 0
    cnt2 = 0
    cnt3 = 0

    fp1 = open('./maogai/Round_1/单选.txt', 'w', encoding='utf-8')
    fp2 = open('./maogai/Round_1/多选.txt', 'w', encoding='utf-8')
    fp3 = open('./maogai/Round_1/判断.txt', 'w', encoding='utf-8')

    times = 0
    for attempt in attempt_list:
        times = times + 1
        new_url = detail_url + attempt
        # print(new_url)
        page_text = session.get(url=new_url, headers=headers)
        tree = etree.HTML(page_text.text)
        pro_list = tree.xpath('//ul[@id="content_listContainer"]/li')

        cnt = 0
        # 每次尝试的内容进行爬取
        for li in pro_list:
            cnt = cnt + 1
            pro_name = li.xpath('normalize-space(.//div[@class="vtbegenerated inlineVtbegenerated"])')
            pro_answer_1 = li.xpath('.//div[@class="vtbegenerated inlineVtbegenerated"]//label/text()')
            pro_answer_1 = str(pro_answer_1)
            pro_answer_2 = li.xpath('normalize-space(.//span[@class="answerTextSpan"])')
            
            if cnt <= 5:
                if pro_name not in danxuan:
                    cnt1 = cnt1 + 1
                    danxuan[pro_name] = pro_answer_1
                    fp1.write(str(cnt1) + '. ' + pro_name+'\n'+pro_answer_1+'\n')
            elif cnt <= 10:
                if pro_name not in duoxuan:
                    cnt2 = cnt2 + 1
                    duoxuan[pro_name] = pro_answer_1
                    fp2.write(str(cnt2) + '. ' + pro_name + '\n' +pro_answer_1+'\n')
            else:
                if pro_name not in panduan:
                    cnt3 = cnt3 +1
                    panduan[pro_name] = pro_answer_2
                    fp3.write(str(cnt3) + '. ' + pro_name + '\n'+pro_answer_2+'\n')
        print('times: ' + str(times) + ' 单选: ' + str(cnt1) + ',多选：' + str(cnt2) + ',判断: ' + str(cnt3) )
   
    fp1.close()
    fp2.close()
    fp3.close()
    print('total: ' + str(cnt1+cnt2+cnt3) + ' problems have been downloaded.')


times: 1 单选: 5,多选：5,判断: 5
times: 2 单选: 10,多选：10,判断: 10
times: 3 单选: 15,多选：13,判断: 14
times: 4 单选: 18,多选：17,判断: 19
times: 5 单选: 21,多选：21,判断: 22
times: 6 单选: 24,多选：23,判断: 25
times: 7 单选: 28,多选：24,判断: 27
times: 8 单选: 30,多选：27,判断: 31
times: 9 单选: 31,多选：29,判断: 33
times: 10 单选: 31,多选：31,判断: 35
times: 11 单选: 32,多选：32,判断: 37
times: 12 单选: 33,多选：35,判断: 38
times: 13 单选: 34,多选：37,判断: 42
times: 14 单选: 35,多选：38,判断: 43
times: 15 单选: 37,多选：40,判断: 44
times: 16 单选: 38,多选：42,判断: 44
times: 17 单选: 39,多选：44,判断: 44
times: 18 单选: 39,多选：47,判断: 44
times: 19 单选: 40,多选：49,判断: 44
times: 20 单选: 40,多选：49,判断: 44
times: 21 单选: 41,多选：49,判断: 44
times: 22 单选: 42,多选：50,判断: 46
times: 23 单选: 42,多选：51,判断: 46
times: 24 单选: 42,多选：51,判断: 47
times: 25 单选: 42,多选：51,判断: 47
times: 26 单选: 43,多选：51,判断: 47
times: 27 单选: 44,多选：51,判断: 48
times: 28 单选: 44,多选：53,判断: 48
times: 29 单选: 46,多选：53,判断: 49
times: 30 单选: 46,多选：53,判断: 51
times: 31 单选: 46,多选：54,判断: 52
times: 32 单选: 47,多选：54,判断: 52
times: 33 单选: 48,多选：54,判断: 53
times: 34 单选: 48,多选：54